In [50]:
from numba import jit, prange
import numpy as np
from scipy.spatial.distance import cdist

In [51]:
X = np.random.random((1000, 6)).astype(float)

In [52]:
%timeit cdist(X, X)
output_scipy = cdist(X, X)

100 loops, best of 3: 9.51 ms per loop


In [53]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M))
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [54]:
%timeit pairwise_python(X)
output = pairwise_python(X)
np.allclose(output, output_scipy)

1 loop, best of 3: 4.95 s per loop


True

In [55]:
@jit(nopython=True)
def pairwise_python_jit(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M))
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [59]:
%timeit pairwise_python_jit(X)
output = pairwise_python(X)
np.allclose(output, output_scipy)

100 loops, best of 3: 7.98 ms per loop


True

In [57]:
@jit(nopython=True, parallel=True)
def pairwise_python_jit_parallel(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M))
    for i in prange(M):  # help the compiler spot an opportunity :)
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [62]:
%timeit pairwise_python_jit_parallel(X)
output = pairwise_python_jit_parallel(X)
np.allclose(output, output_scipy)

100 loops, best of 3: 2.89 ms per loop


True